In [1]:
###Required packages
import numpy as np
from hyperopt import hp, fmin, tpe
from scikits.odes.ode import ode
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec
from pyomo.environ import *
import time
import random
from scipy.optimize import Bounds, minimize



In [2]:
def toy_model(t, y, ydot, params):
    kc=12.; km=10.; lam=1.93E-4; Vin=1.; e0=0.0467
    T = 1; E = 2; X = 2
    A, W = params
    ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
    ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
    for e in range(E):
        ydot[e+X] = -lam*y[e+X] + np.sum(A[e]*np.array([activation(y[T], W[e][2], W[e][1], W[e][0]), repression(y[T], W[e][2], W[e][1], W[e][0]), W[e][2]]))
    ydot[E+X] = (Vin - y[X+1]*nonlinearity(y[X-1], kc, km))**2 #J1
    ydot[E+X+1] = np.sum([np.sum(A[e]*np.array([activation(y[T], W[e][2], W[e][1], W[e][0]), repression(y[T], W[e][2], W[e][1], W[e][0]), W[e][2]])) for e in range(E)]) #J2

def arch_converter(A):
    if A == 'Dual Control':
        return ((0, 1, 0), (1, 0, 0))

    elif A == 'No Control':
        return ((0, 0, 1), (0, 0, 1))

    elif A == 'Downstream Activation':
        return ((0, 0, 1), (1, 0, 0))

    elif A == 'Upstream Repression':
        return ((0, 1, 0), (0, 0, 1))
    else: return 'Invalid Circuit'

def name_converter(A):
    if A == ((0, 1, 0), (1, 0, 0)):
        return 'Dual Control'

    elif A == ((0, 0, 1), (0, 0, 1)):
        return 'No Control'

    elif A == ((0, 0, 1), (1, 0, 0)):
        return 'Downstream Activation'

    elif A == ((0, 1, 0), (0, 0, 1)):
        return 'Upstream Repression'
    else: return 'Invalid Circuit'

def loss_biological(j1, j2, alpha1=1E-5, alpha2=1E-2):
        """Computes scalarized loss including genetic constraint and product production"""
        loss = alpha1*j1 + alpha2*j2
        return j1, j2, loss

def activation(x, k, theta, n):
    return (k*(x/theta)**n)/(1+(x/theta)**n)

def repression(x, k, theta, n):
    return k/(1+(x/theta)**n)
    
def nonlinearity(x, kc, km):
    return (kc*x)/(km+x)

In [64]:
def ode_fun(x, function):
    k1, k2, theta1, theta2 = x
    t = np.linspace(0, 5E4, 100) 
    y0 = np.array([2290., 0., 0., 0., 0., 0.])
    architecture = arch_converter(function)
    param_values = [[2., theta1, k1], [2., theta2, k2]]
    extra_options = {'old_api': False, 'user_data': [architecture, param_values]} #'rtol':1E-3
    ode_solver = ode('cvode', toy_model, **extra_options)
    solution = ode_solver.solve(t, y0)
    j1, j2 = solution.values.y[-1, -2:]
    j1, j2, loss = loss_biological(j1, j2, alpha1=1E-5, alpha2=1E-2)
    row = [k1, k2, theta1, theta2, loss, j2, j2, str(solution.flag)]
    trace.loc[len(trace)] = row
    if (str(solution.flag) == 'StatusEnum.SUCCESS') and loss > 0.0: return loss
    else: return 1.E7


columns = ['k1', 'k2', 'theta1', 'theta2', 'loss', 'j1', 'j2', 'sol_flag']
trace = pd.DataFrame(columns=columns)
bounds = Bounds([1E-7, 1E-7, 0.001, 0.001], [0.001, 0.001, 10., 10.])

start_time = time.time()
function = 'Dual Control'
init_values = [np.random.uniform(1E-7, 1E-3), np.random.uniform(1E-7, 1E-3), np.random.uniform(0.001, 10.), np.random.uniform(0.001, 10.)]
sol = minimize(ode_fun, init_values, args=(function), method='TNC', bounds=bounds) #, options={'gtol':1e-10, 'stepmx':1E-2}
elapsed_time = time.time() - start_time
sol

     fun: 0.07006312840184184
     jac: array([ -0.52316399, -15.55662587,   4.4913418 ,   0.18073589])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 150
     nit: 4
  status: 1
 success: True
       x: array([5.30351116e-04, 2.07194316e-05, 6.66666171e+00, 7.69816386e-01])

In [73]:
solutions = []
times = []
traces = []['Dual Control', 'Upstream Repression', 'Downstream Activation', 'No Control']
for j in :
    for i in range(100):
        start_time = time.time()
        columns = ['k1', 'k2', 'theta1', 'theta2', 'loss', 'j1', 'j2', 'sol_flag']
        trace = pd.DataFrame(columns=columns)
        bounds = Bounds([1E-7, 1E-7, 0.001, 0.001], [0.001, 0.001, 10., 10.])

        function = j
        init_values = [np.random.uniform(1E-7, 1E-3), np.random.uniform(1E-7, 1E-3), np.random.uniform(0.001, 10.), np.random.uniform(0.001, 10.)]
        sol = minimize(ode_fun, init_values, args=(function), method='TNC', bounds=bounds) #, options={'gtol':1e-10, 'stepmx':1E-2}
        elapsed_time = time.time() - start_time
        times.append(elapsed_time)
        solutions.append(sol)
        traces.append(trace)
        print(i)
    print('Completed Architecture', j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Completed Architecture Dual Control
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Completed Architecture Upstream Repression
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Completed Architecture Downstream Activation
0
1
2


In [74]:
visualization_data = pd.DataFrame({'Method':[], 'Computational Time':[], 'Best Loss':[], 'Function Evaluations':[], 'Message':[]})
for s, t in zip(solutions, times):
    row = pd.DataFrame({'Method': ['Gradient-Based'], 'Computational Time': [t], 'Best Loss': [s.fun], 'Function Evaluations': [s.nfev], 'Message':[s.message]})
    visualization_data = pd.concat([visualization_data, row])
visualization_data.to_csv('nlp_100_fixed.csv')